# USGS Earthquake Data with pystac-monty

This notebook demonstrates how to use pystac-monty to process USGS earthquake data and visualize it using interactive maps. We'll:

1. Fetch the latest major earthquakes from USGS
2. Convert them to STAC items using pystac-monty
3. Display events on an interactive map
4. Allow selection of events to view related hazards and impacts
5. Explore the Monty STAC model and its metadata

Let's begin by importing the necessary libraries.

In [ ]:
# Basic libraries
import os
import json
import requests
from datetime import datetime, timedelta
import pytz
import pandas as pd
import numpy as np
from pathlib import Path

# STAC and pystac-monty
import pystac
from pystac_monty.extension import MontyExtension, HazardDetail, ImpactDetail, MontyEstimateType
from pystac_monty.sources.usgs import USGSDataSource, USGSTransformer
from pystac_monty.geocoding import WorldAdministrativeBoundariesGeocoder as MontyGeoCoder
from pystac_monty.hazard_profiles import MontyHazardProfiles

# Visualization libraries
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

## 1. Fetch Recent Major Earthquakes from USGS

Let's fetch earthquakes with magnitude 6.0+ from the past 30 days using the USGS API.

In [ ]:
# Define USGS API endpoint for fetching earthquake data
def fetch_usgs_earthquakes(min_magnitude=6.0, days=30):
    """
    Fetch earthquake data from USGS API
    
    Parameters:
    - min_magnitude: Minimum magnitude to filter earthquakes (default: 6.0)
    - days: Number of days to look back (default: 30)
    
    Returns:
    - List of earthquake data as dictionaries
    """
    # Calculate the start time (days ago from now)
    start_time = datetime.now() - timedelta(days=days)
    start_time_str = start_time.strftime("%Y-%m-%dT%H:%M:%S")
    
    # USGS earthquake API endpoint
    url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
    
    # Parameters for the API request
    params = {
        "format": "geojson",
        "starttime": start_time_str,
        "minmagnitude": min_magnitude,
        "orderby": "time"
    }
    
    # Make the request to the USGS API
    response = requests.get(url, params=params)
    data = response.json()
    
    print(f"Found {len(data['features'])} earthquakes with magnitude {min_magnitude}+ in the last {days} days")
    return data

# Fetch recent major earthquakes
earthquake_data = fetch_usgs_earthquakes(min_magnitude=6.0, days=30)
earthquake_features = earthquake_data['features']

In [ ]:
# Extract relevant information into a DataFrame
earthquakes_df = pd.DataFrame([
    {
        'id': eq['id'],
        'title': eq['properties']['title'],
        'time': datetime.fromtimestamp(eq['properties']['time']/1000, pytz.UTC),
        'magnitude': eq['properties']['mag'],
        'place': eq['properties']['place'],
        'longitude': eq['geometry']['coordinates'][0],
        'latitude': eq['geometry']['coordinates'][1],
        'depth': eq['geometry']['coordinates'][2],
        'tsunami': bool(eq['properties'].get('tsunami')),
        'felt': eq['properties'].get('felt') or 0
    } for eq in earthquake_features
])

# Sort by magnitude (descending)
earthquakes_df = earthquakes_df.sort_values(by='magnitude', ascending=False)

# Display the DataFrame
earthquakes_df

## 2. Creating Synthetic STAC Items

Let's create synthetic STAC items from the earthquake data using the pystac-monty extension.

In [ ]:
# Initialize the hazard profiles
hazard_profiles = MontyHazardProfiles()

# Function to create synthetic STAC items from earthquake data
def create_synthetic_stac_items(earthquake_features):
    all_items = []
    
    for feature in earthquake_features:
        event_id = feature['id']
        props = feature['properties']
        coords = feature['geometry']['coordinates']
        
        # Create a point geometry
        geometry = {
            "type": "Point",
            "coordinates": [coords[0], coords[1]]
        }
        
        # Create event item
        event_item = pystac.Item(
            id=f"usgs-event-{event_id}",
            geometry=geometry,
            bbox=[coords[0], coords[1], coords[0], coords[1]],
            datetime=datetime.fromtimestamp(props['time']/1000, pytz.UTC),
            properties={
                "title": props['title'],
                "description": props['place'],
                "eq:magnitude": props['mag'],
                "eq:magnitude_type": props.get('magType', 'mb'),
                "eq:depth": coords[2],
                "eq:status": props.get('status', 'reviewed'),
                "eq:tsunami": bool(props.get('tsunami', 0)),
                "eq:felt": props.get('felt', 0),
                "roles": ["source", "event"]
            }
        )
        
        # Add Monty extension
        MontyExtension.add_to(event_item)
        monty = MontyExtension.ext(event_item)
        monty.episode_number = 1
        monty.hazard_codes = ["GH0004"]  # Earthquake code
        monty.country_codes = ["USA"]  # Placeholder - in reality this would be geocoded
        monty.correlation_id = f"20250101T000000Z-{event_id}-GEO-SEIS-001-USGS"
        
        # Add source link
        event_item.add_link(pystac.Link(
            "via", 
            f"https://earthquake.usgs.gov/earthquakes/eventpage/{event_id}/executive", 
            "application/json", 
            "USGS Event Data"
        ))
        
        all_items.append(event_item)
        
        # Create hazard item (shakemap)
        hazard_item = event_item.clone()
        hazard_item.id = f"usgs-hazard-{event_id}-shakemap"
        
        # Update the bbox to represent a larger area (shakemap extent)
        # Just extend by ~1 degree in each direction for demonstration
        hazard_item.bbox = [coords[0]-1, coords[1]-1, coords[0]+1, coords[1]+1]
        hazard_item.geometry = {
            "type": "Polygon",
            "coordinates": [[
                [coords[0]-1, coords[1]-1],
                [coords[0]+1, coords[1]-1],
                [coords[0]+1, coords[1]+1],
                [coords[0]-1, coords[1]+1],
                [coords[0]-1, coords[1]-1]
            ]]
        }
        
        hazard_item.properties["roles"] = ["source", "hazard"]
        hazard_item.properties["title"] = f"ShakeMap for {props['title']}"
        
        # Add hazard detail to Monty extension
        monty = MontyExtension.ext(hazard_item)
        monty.hazard_detail = HazardDetail(
            cluster="GEO-SEIS",
            severity_value=float(props['mag']),
            severity_unit=props.get('magType', 'mb'),
            estimate_type=MontyEstimateType.PRIMARY
        )
        
        all_items.append(hazard_item)
        
        # Only create impact items for stronger earthquakes (M6.5+)
        if props['mag'] >= 6.5:
            # Create fatalities impact item
            impact_item = event_item.clone()
            impact_item.id = f"usgs-impact-{event_id}-fatalities-USA"
            impact_item.properties["roles"] = ["source", "impact"]
            impact_item.properties["title"] = f"Estimated Fatalities for {props['title']}"
            
            # Synthetic impact value based on magnitude
            estimated_fatalities = int((props['mag'] - 6.0) * 10) 
            
            # Add impact detail to Monty extension
            monty = MontyExtension.ext(impact_item)
            monty.impact_detail = ImpactDetail(
                category="people",
                type="death",
                value=float(estimated_fatalities),
                unit="people",
                estimate_type=MontyEstimateType.MODELLED
            )
            
            # Use the same geometry as the hazard
            impact_item.geometry = hazard_item.geometry 
            impact_item.bbox = hazard_item.bbox
            
            all_items.append(impact_item)
            
            # Create economic impact item
            impact_item = event_item.clone()
            impact_item.id = f"usgs-impact-{event_id}-economic-USA"
            impact_item.properties["roles"] = ["source", "impact"]
            impact_item.properties["title"] = f"Estimated Economic Losses for {props['title']}"
            
            # Synthetic impact value based on magnitude 
            estimated_losses = int((props['mag'] - 6.0) * 1000000)
            
            # Add impact detail to Monty extension
            monty = MontyExtension.ext(impact_item)
            monty.impact_detail = ImpactDetail(
                category="buildings",
                type="cost",
                value=float(estimated_losses),
                unit="usd",
                estimate_type=MontyEstimateType.MODELLED
            )
            
            # Use the same geometry as the hazard
            impact_item.geometry = hazard_item.geometry
            impact_item.bbox = hazard_item.bbox
            
            all_items.append(impact_item)
    
    return all_items

# Create synthetic STAC items
all_stac_items = create_synthetic_stac_items(earthquake_features)
print(f"Created {len(all_stac_items)} STAC items from {len(earthquake_features)} earthquakes")

In [ ]:
# Separate the STAC items by role
event_items = []
hazard_items = []
impact_items = []

for item in all_stac_items:
    roles = item.properties.get('roles', [])
    if 'event' in roles:
        event_items.append(item)
    elif 'hazard' in roles:
        hazard_items.append(item)
    elif 'impact' in roles:
        impact_items.append(item)

print(f"Events: {len(event_items)}, Hazards: {len(hazard_items)}, Impacts: {len(impact_items)}")

## 3. Displaying Earthquakes on a Map

Now, let's create an interactive map to display the earthquakes.

In [ ]:
# Function to create a map of earthquakes
def create_earthquake_map(event_items):
    # Calculate the average latitude and longitude for map centering
    if not event_items:
        return folium.Map(location=[0, 0], zoom_start=2)
    
    lats = [item.bbox[1] for item in event_items]
    lons = [item.bbox[0] for item in event_items]
    center_lat = sum(lats) / len(lats)
    center_lon = sum(lons) / len(lons)
    
    # Create the map
    m = folium.Map(location=[center_lat, center_lon], zoom_start=2, tiles='CartoDB positron')
    
    # Add a title
    title_html = '''
    <h3 align="center" style="font-size:20px">
        <b>Recent Major Earthquakes (M6.0+)</b>
    </h3>
    '''
    m.get_root().html.add_child(folium.Element(title_html))
    
    # Create a marker cluster group
    marker_cluster = MarkerCluster(name="Earthquakes").add_to(m)
    
    # Add markers for each earthquake
    for item in event_items:
        # Get the earthquake properties
        mag = item.properties.get('eq:magnitude')
        title = item.properties.get('title')
        date_time = item.datetime.strftime('%Y-%m-%d %H:%M:%S UTC')
        
        # Calculate marker size and color based on magnitude
        size = max(6, mag * 3)  # Scale marker size
        
        # Color based on magnitude
        if mag >= 8.0:
            color = 'darkred'
        elif mag >= 7.0:
            color = 'red'
        elif mag >= 6.5:
            color = 'orange'
        else:
            color = 'lightred'
        
        # Create popup content
        popup_content = f"""
        <b>{title}</b><br>
        <b>Magnitude:</b> {mag}<br>
        <b>Time:</b> {date_time}<br>
        <b>ID:</b> {item.id}<br>
        <b>Depth:</b> {item.properties.get('eq:depth')} km<br>
        """
        
        # Add marker to the cluster
        folium.CircleMarker(
            location=[item.bbox[1], item.bbox[0]],
            radius=size,
            color=color,
            fill=True,
            fill_opacity=0.7,
            popup=folium.Popup(popup_content, max_width=300),
            tooltip=f"M{mag} - {title}"
        ).add_to(marker_cluster)
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    return m

# Create the earthquake map
earthquake_map = create_earthquake_map(event_items)
earthquake_map